In [15]:
import kaggle
import os

# This confirms the Kaggle API is available
print("Kaggle API imported successfully.")

Kaggle API imported successfully.


In [16]:
# Clean up any old zip file first
if os.path.exists("retail-orders.zip"):
    os.remove("retail-orders.zip")
    print("Removed old zip file.")

# Now, force the download
!kaggle datasets download ankitbansal06/retail-orders --force


  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|██████████| 200k/200k [00:00<00:00, 56.7MB/s]


Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0



In [20]:
#extract file from zip file
import zipfile
zip_ref = zipfile.ZipFile('retail-orders.zip')
zip_ref.extractall()    #extract file to dir
zip_ref.close()    #close the file

In [24]:
#read data from the file4 and handle null values
import pandas as pd
df = pd.read_csv('orders.csv', sep=',', encoding='latin-1', on_bad_lines='skip',
                 engine='python', na_values=['Not Available','unknown']) #changes unknown values to nan
df['Ship Mode'].unique()   #returns unique values from the column

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [35]:
#rename column names ...make them lower case and replace them with underscore
#df.replace(columns= {'Order ID':'order_id'})  #this consumes more time hance we will approach another method
#df.columns=df.columns.str.lower()  #extracted -> converted to str -> applied lower()
df.columns=df.columns.str.replace(' ', '_')   #replaced spaces with underscore
df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [40]:
#derive new columns discount, sale price and profit
#df['discount']=df['list_price']*df['discount_percent']*.01
#df['sale_price']=df['list_price']-df['discount']
df['profit']=df['sale_price']-df['cost_price']
df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-1.0


In [45]:
#convert order date from object datatype to datetime
df['order_date']=pd.to_datetime(df['order_date'], format='%Y-%m-%d')    #change the dtype of column str to datetime

In [48]:
#drop the columns we dont need
df.drop(columns=['list_price', 'cost_price', 'discount_percent'], inplace=True)

In [56]:
#Load the data into SQL server using replace option
import sqlalchemy as sal
from urllib.parse import quote_plus # <--- NEW IMPORT for safety

DB_USERNAME = "root"
DB_PASSWORD = "Buck@#123"
DB_HOST = "localhost"
DB_PORT = "3306"
DB_NAME = "retail_order"
# 1. Safely URL-encode the password to handle special characters
encoded_password = quote_plus(DB_PASSWORD)

# 2. Constructing the URL with the robust mysql+mysqldb driver
# We change 'mysql+pymysql' to 'mysql+mysqldb' and use the encoded password
connection_url = (
    f"mysql+mysqldb://{DB_USERNAME}:{encoded_password}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

# Creating the engine
try:
    engine = sal.create_engine(connection_url)

    # Test connection
    with engine.connect() as connection:
        print("✅ Engine created and tested with the robust mysqlclient driver.")

except Exception as e:
    print(f"❌ Connection Error: {e}")
    print("Please ensure your MySQL server is running and the 'mysqlclient' driver is installed.")

✅ Engine created and tested with the robust mysqlclient driver.


In [60]:
#Load the data into MySQL using append option
TABLE_NAME = 'df_order' # Name of the new table in your MySQL database

try:
    df.to_sql(
        name=TABLE_NAME,
        con=engine,
        if_exists='append', # Drops the table and recreates it every time
        index=False,         # Prevents the Pandas index from being uploaded as a column
        chunksize=1000       # Uploads data in batches for better performance
    )

    print(f"SUCCESS! Data loaded into MySQL table: '{TABLE_NAME}'")

except Exception as e:
    print(f"\n❌ FATAL UPLOAD ERROR: The data upload failed. Error details:")
    print(e)

SUCCESS! Data loaded into MySQL table: 'df_order'
